# Mutations
In this notebook we will focus on each of the mutatons individually.

In [1]:
%matplotlib inline
from scipy.stats import pearsonr
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.decomposition import NMF, PCA
from sklearn.manifold import TSNE

from source import load_avenio_files
from transform import dummy_encode_mutations, get_top_correlated, mutation_train_test_split, patient_allele_frequencies


RANDOM_STATE = 1234
np.random.seed(RANDOM_STATE)

In [2]:
# Load data from spreadsheet and SPSS files.
mutation_data_frame, phenotypes = load_avenio_files()

Two ways to calculate the differences:
- Using the allele frequencies.
- Using the mutatant concentration.

In [3]:
# Vocabulary is the entire dataset, not only training set. Otherwise we run into problems during inference.
gene_vocabulary = mutation_data_frame['Gene'].unique()
# Columns containing allele frequencies.
allele_columns = ["T0: Allele \nFraction", "T1: Allele Fraction"]
# # Columns containing mutant concentration.
# allele_columns = ["T0: No. Mutant \nMolecules per mL", "T1: No. Mutant \nMolecules per mL"]

# 1) Convert to float.
columns_to_numeric = allele_columns
for column_name in allele_columns:
    mutation_data_frame.loc[:, column_name] = pd.to_numeric(mutation_data_frame[column_name], errors='coerce')
# 2) Drop rows for which the columns can not be converted.
mutation_data_frame = mutation_data_frame.dropna(subset=allele_columns)

Calculate the r-fraction as $$r = \frac{\Delta t}{t_0} \equiv \frac{t_1 - t_0}{t_0} .$$

In [26]:
def r(t_0, t_1):
    return (t_1 - t_0) / t_0

In [27]:
patient_mutation_frequencies = patient_allele_frequencies(
    mutation_data_frame, 
    gene_vocabulary, 
    # Calculate r(t_0, t_1).
    r,
    # Sum mutation values per gene in each patient.
    handle_duplicates="sum",
)

# Correlations
How do gene's fractions correlate? We calculate the Pearson correlation value which is defined as:
$$C_{ij} = \sum_{m=1}^{N} \frac{(X_{mi} - \mu_i)(X_{mj} - \mu_j)}{\sigma_i \sigma_j} \, ,$$
with $\mu$ and $\sigma$ the mean and standard deviation, respectively.

In [28]:
def pearson_pval(x, y):
    return pearsonr(x, y)[1]

In [29]:
corr = patient_mutation_frequencies.corr().fillna(0)
pval_corr = patient_mutation_frequencies.corr(method=pearson_pval).fillna(1)
corr.style.background_gradient(cmap='coolwarm', axis=None)

### Negative correlation

In [30]:
gene_counts = mutation_data_frame['Gene'].value_counts()
get_top_correlated(
    corr, 
    pval_corr,
    gene_counts=gene_counts, 
    ascending=True, 
    top_count=4,
)

,gene 1,gene 2,correlation,p-value,# gene 1,# gene 2
0,NFE2L2,KDR,-0.583724,6.341728e-11,9.0,1.0
2,PTEN,CDKN2A,-0.333554,5.067958e-04,4.0,7.0
4,KRAS,SMO,-0.078985,4.231809e-01,37.0,1.0
6,TP53,PTEN,-0.050080,6.119164e-01,100.0,4.0


The p-values should not be taken to seriously. The fact that the p-values are extremely low is easy to understand:
- All columns are zero.
- Except the columns where the two unique mutations happen to coincide.
This immediately implies that the p-value should be near zero.

### Positive correlation

In [31]:
get_top_correlated(corr, pval_corr, gene_counts=gene_counts, ascending=False)

,gene 1,gene 2,correlation,p-value,# gene 1,# gene 2
1762,MTOR,GATA3,1.000000,0.000000e+00,1.0,1.0
1726,CD274,MAP2K2,1.000000,0.000000e+00,1.0,1.0
1724,SMAD4,CTNNB1,0.999821,2.902454e-179,2.0,2.0
1722,STK11,GNAS,0.929957,1.470575e-46,4.0,1.0
1720,PTEN,FGFR1,0.660249,1.833417e-14,4.0,1.0
1718,JAK2,PIK3CA,0.534706,4.215318e-09,1.0,7.0
1716,FBXW7,PTEN,0.401111,2.230812e-05,4.0,4.0
1714,APC,KEAP1,0.396476,2.827031e-05,2.0,4.0
1712,SMO,STK11,0.375571,7.883190e-05,1.0,4.0
1710,TP53,KRAS,0.375138,8.046542e-05,100.0,37.0


## Do responders show an increase in mutational allel frequency?

Calculate quantities to comapre.

In [ ]:
mutation_data_frame['f_t'] = f_t(mutation_data_frame[allele_columns[0]], mutation_data_frame[allele_columns[1]])
mutation_data_frame['dt'] = mutation_data_frame[allele_columns[1]] - mutation_data_frame[allele_columns[0]]
mutation_data_frame['ratio'] = mutation_data_frame[allele_columns[1]] / mutation_data_frame[allele_columns[0]]

Add phenotype data.

In [ ]:
mutation_data_frame['response'] = mutation_data_frame['Patient ID'].apply(lambda x: phenotypes.loc[x, 'response_grouped'])
mutation_data_frame['progression'] = mutation_data_frame['Patient ID'].apply(lambda x: phenotypes.loc[x, 'progressie'])

In [ ]:
gene_subset = mutation_data_frame['Gene'].isin(['TP53', 'KRAS', 'PIK3CA', 'NFE2L2'])

g = sns.catplot(
    x='Gene', 
    y='f_t', 
    hue='response',
    data=mutation_data_frame[gene_subset],
    kind='violin',
)
g.fig.set_size_inches(16,8)

Absolute difference

In [ ]:
g = sns.catplot(
    x='Gene', 
    y='dt', 
    hue='response', 
#     col='progression', 
    data=mutation_data_frame[gene_subset],
#     col_wrap=1,
    kind='violin',
)
# g.ax.set_ylim([-.5, .5])
g.fig.set_size_inches(16,8)

In [ ]:
g = sns.catplot(
    x='Gene', 
    y='ratio', 
    hue='response', 
#     col='progression', 
    data=mutation_data_frame[gene_subset], 
    kind='violin',
)
g.fig.set_size_inches(16,8)
# g.ax.set_ylim([-.5, .5])